In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
from keras.layers import LSTM, Dense, SimpleRNN
import tensorflow as tf
import statistics
import keras
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
%load_ext tensorboard

In [2]:
data_path_and_labels = pd.read_csv("../mapped_data/xalan-2.5.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xalan-2.5/xalan-j_2_5_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [3]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.5/xalan-j_2_5_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [4]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.5/xalan-j_2_5_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent'

In [5]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [6]:
d = pd.read_csv("../mapped_data/xalan-2.5.csv")
d['data_path'][0]

'dataset/promise/defect data/xalan-2.5/xalan-j_2_5_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [7]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/xalan-2.5/xalan-j_2_5_0/src/org/w3c/dom/xpath/XPathNamespace.java.restrictedcontent.embed'

In [8]:
d

data_path   bugs
0    ../dataset/promise/defect data/xalan-2.5/xalan...      0
1    ../dataset/promise/defect data/xalan-2.5/xalan...      0
2    ../dataset/promise/defect data/xalan-2.5/xalan...      0
3    ../dataset/promise/defect data/xalan-2.5/xalan...      0
4    ../dataset/promise/defect data/xalan-2.5/xalan...      0
..                                                 ...    ...
749  ../dataset/promise/defect data/xalan-2.5/xalan...      0
750  ../dataset/promise/defect data/xalan-2.5/xalan...      0
751  ../dataset/promise/defect data/xalan-2.5/xalan...      1
752  ../dataset/promise/defect data/xalan-2.5/xalan...      0
753  ../dataset/promise/defect data/xalan-2.5/xalan...      0

[754 rows x 2 columns]

In [9]:
train, test = train_test_split(d, test_size=0.2)

In [10]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [11]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [12]:
train_x[0]

[2407.0,
 184.0,
 184.0,
 3573.0,
 6713.0,
 277.0,
 6706.0,
 4848.0,
 1334.0,
 2450.0,
 2450.0,
 8114.0,
 1022.0,
 573.0,
 8114.0,
 1022.0,
 573.0,
 153.0,
 8114.0,
 1022.0,
 153.0,
 8114.0,
 1022.0,
 153.0,
 8114.0,
 1022.0,
 573.0,
 153.0,
 8114.0,
 1022.0,
 2578.0,
 8114.0,
 1022.0,
 153.0,
 8114.0,
 1022.0,
 153.0,
 8114.0,
 1022.0,
 153.0,
 8114.0,
 1022.0,
 1130.0,
 8114.0,
 1022.0,
 1130.0,
 8114.0,
 1022.0,
 1130.0,
 8114.0,
 1022.0,
 1130.0,
 8114.0,
 1022.0,
 1130.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 1878.0,
 8114.0,
 1022.0,
 2109.0,
 38.0,
 8114.0,
 1022.0,
 2109.0,
 38.0,
 8114.0,
 1022.0,
 2109.0,
 38.0,
 8114.0,
 1022.0,
 2109.0,
 38.0,
 8114.0,
 1022.0,
 2109.0,
 38.0,
 573.0,
 153.0,
 8114.0,
 1022.0,
 153.0,
 8114.0,
 1022.0,
 153.0,
 154.0,
 1

In [13]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

9675.0

In [14]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [15]:
train_y

array([4, 0, 0, 2, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 2, 1, 1, 0, 1, 0, 0, 0, 1, 1, 3, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 2, 0, 1, 0, 0, 1, 2, 0, 0, 1, 2, 1, 1, 1, 0, 0, 3, 1, 0, 2, 2,
       1, 2, 1, 4, 4, 1, 0, 0, 1, 0, 0, 1, 0, 5, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 9, 0, 1, 1, 1, 1, 0, 1, 2, 2, 0, 0, 1,
       0, 0, 1, 0, 0, 2, 6, 1, 1, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 2, 1, 0, 0, 1, 1, 2, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 2, 1, 1, 1, 1, 1, 1, 0, 2, 1, 4, 1, 0, 1, 1, 1, 0,
       1, 2, 1, 0, 0, 0, 1, 0, 2, 1, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 2, 0, 1, 0, 0, 0, 1, 1,
       0, 2, 1, 1, 0, 1, 0, 3, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       2, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,

In [16]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [17]:
train_x.shape

(603, 7389)

In [18]:
def define_model():
    # define the model
    model = Sequential()
    model.add(tf.keras.layers.Embedding(10671, 6, input_length=train_x.shape[1]))
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    # compile the model
    model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )  
    return model

In [19]:
y = list(test_y)
y

[0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 3,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 2,
 1,
 0,
 0,
 4,
 3,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 3,
 1,
 1,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0]

In [20]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
total_predictions = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/AST_Training_curves/5dim_xalan2.5/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [round(i) for i in x]
    total_predictions.append(x)
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
7/7 [==============================] - 1s 84ms/step - loss: 1.4573 - mse: 1.4573 - mae: 0.7111 - root_mean_squared_error: 1.2072 - mean_squared_logarithmic_error: 0.3753
Epoch 2/100
7/7 [==============================] - 0s 58ms/step - loss: 1.4076 - mse: 1.4076 - mae: 0.7091 - root_mean_squared_error: 1.1864 - mean_squared_logarithmic_error: 0.3493
Epoch 3/100
7/7 [==============================] - 0s 72ms/step - loss: 1.3360 - mse: 1.3360 - mae: 0.7075 - root_mean_squared_error: 1.1559 - mean_squared_logarithmic_error: 0.3133
Epoch 4/100
7/7 [==============================] - 0s 63ms/step - loss: 1.2365 - mse: 1.2365 - mae: 0.7046 - root_mean_squared_error: 1.1120 - mean_squared_logarithmic_error: 0.2698
Epoch 5/100
7/7 [==============================] - 1s 73ms/step - loss: 1.1296 - mse: 1.1296 - mae: 0.7024 - root_mean_squared_error: 1.0628 - mean_squared_logarithmic_error: 0.2319
Epoch 6/100
7/7 [==============================] - 0s 69ms/step - loss: 1.0244 -

7/7 [==============================] - 1s 71ms/step - loss: 0.6877 - mse: 0.6877 - mae: 0.6362 - root_mean_squared_error: 0.8292 - mean_squared_logarithmic_error: 0.1811
Epoch 45/100
7/7 [==============================] - 1s 77ms/step - loss: 0.6797 - mse: 0.6797 - mae: 0.6325 - root_mean_squared_error: 0.8244 - mean_squared_logarithmic_error: 0.1754
Epoch 46/100
7/7 [==============================] - 1s 77ms/step - loss: 0.6734 - mse: 0.6734 - mae: 0.6294 - root_mean_squared_error: 0.8206 - mean_squared_logarithmic_error: 0.1715
Epoch 47/100
7/7 [==============================] - 1s 71ms/step - loss: 0.6674 - mse: 0.6674 - mae: 0.6270 - root_mean_squared_error: 0.8169 - mean_squared_logarithmic_error: 0.1691
Epoch 48/100
7/7 [==============================] - 1s 91ms/step - loss: 0.6613 - mse: 0.6613 - mae: 0.6256 - root_mean_squared_error: 0.8132 - mean_squared_logarithmic_error: 0.1689
Epoch 49/100
7/7 [==============================] - 1s 102ms/step - loss: 0.6539 - mse: 0.6539 - m

7/7 [==============================] - 1s 91ms/step - loss: 0.4888 - mse: 0.4888 - mae: 0.5593 - root_mean_squared_error: 0.6992 - mean_squared_logarithmic_error: 0.1448
Epoch 89/100
7/7 [==============================] - 1s 97ms/step - loss: 0.4861 - mse: 0.4861 - mae: 0.5581 - root_mean_squared_error: 0.6972 - mean_squared_logarithmic_error: 0.1446
Epoch 90/100
7/7 [==============================] - 1s 89ms/step - loss: 0.4826 - mse: 0.4826 - mae: 0.5585 - root_mean_squared_error: 0.6947 - mean_squared_logarithmic_error: 0.1456
Epoch 91/100
7/7 [==============================] - 1s 95ms/step - loss: 0.4795 - mse: 0.4795 - mae: 0.5583 - root_mean_squared_error: 0.6925 - mean_squared_logarithmic_error: 0.1464
Epoch 92/100
7/7 [==============================] - 1s 92ms/step - loss: 0.4770 - mse: 0.4770 - mae: 0.5570 - root_mean_squared_error: 0.6906 - mean_squared_logarithmic_error: 0.1461
Epoch 93/100
7/7 [==============================] - 1s 96ms/step - loss: 0.4742 - mse: 0.4742 - ma

7/7 [==============================] - 1s 92ms/step - loss: 0.9302 - mse: 0.9302 - mae: 0.6846 - root_mean_squared_error: 0.9645 - mean_squared_logarithmic_error: 0.2160
Epoch 31/100
7/7 [==============================] - 1s 71ms/step - loss: 0.9309 - mse: 0.9309 - mae: 0.6842 - root_mean_squared_error: 0.9648 - mean_squared_logarithmic_error: 0.2221
Epoch 32/100
7/7 [==============================] - 1s 75ms/step - loss: 0.9214 - mse: 0.9214 - mae: 0.6820 - root_mean_squared_error: 0.9599 - mean_squared_logarithmic_error: 0.2153
Epoch 33/100
7/7 [==============================] - 0s 41ms/step - loss: 0.9232 - mse: 0.9232 - mae: 0.6826 - root_mean_squared_error: 0.9609 - mean_squared_logarithmic_error: 0.2005
Epoch 34/100
7/7 [==============================] - 0s 42ms/step - loss: 0.9321 - mse: 0.9321 - mae: 0.6835 - root_mean_squared_error: 0.9654 - mean_squared_logarithmic_error: 0.1961
Epoch 35/100
7/7 [==============================] - 0s 51ms/step - loss: 0.9187 - mse: 0.9187 - ma

7/7 [==============================] - 1s 97ms/step - loss: 0.6889 - mse: 0.6889 - mae: 0.6233 - root_mean_squared_error: 0.8300 - mean_squared_logarithmic_error: 0.1671
Epoch 75/100
7/7 [==============================] - 1s 95ms/step - loss: 0.6857 - mse: 0.6857 - mae: 0.6280 - root_mean_squared_error: 0.8281 - mean_squared_logarithmic_error: 0.1741
Epoch 76/100
7/7 [==============================] - 1s 94ms/step - loss: 0.6769 - mse: 0.6769 - mae: 0.6258 - root_mean_squared_error: 0.8227 - mean_squared_logarithmic_error: 0.1728
Epoch 77/100
7/7 [==============================] - 1s 88ms/step - loss: 0.6711 - mse: 0.6711 - mae: 0.6242 - root_mean_squared_error: 0.8192 - mean_squared_logarithmic_error: 0.1716
Epoch 78/100
7/7 [==============================] - 1s 93ms/step - loss: 0.6699 - mse: 0.6699 - mae: 0.6267 - root_mean_squared_error: 0.8185 - mean_squared_logarithmic_error: 0.1755
Epoch 79/100
7/7 [==============================] - 1s 93ms/step - loss: 0.6621 - mse: 0.6621 - ma

...

7/7 [==============================] - 0s 36ms/step - loss: 1.2414 - mse: 1.2414 - mae: 0.7058 - root_mean_squared_error: 1.1142 - mean_squared_logarithmic_error: 0.2705
Epoch 4/100
7/7 [==============================] - 0s 35ms/step - loss: 1.1202 - mse: 1.1202 - mae: 0.7000 - root_mean_squared_error: 1.0584 - mean_squared_logarithmic_error: 0.2279
Epoch 5/100
7/7 [==============================] - 0s 35ms/step - loss: 1.0259 - mse: 1.0259 - mae: 0.7002 - root_mean_squared_error: 1.0128 - mean_squared_logarithmic_error: 0.2084
Epoch 6/100
7/7 [==============================] - 0s 36ms/step - loss: 0.9659 - mse: 0.9659 - mae: 0.6948 - root_mean_squared_error: 0.9828 - mean_squared_logarithmic_error: 0.2085
Epoch 7/100
7/7 [==============================] - 0s 35ms/step - loss: 0.9502 - mse: 0.9502 - mae: 0.6912 - root_mean_squared_error: 0.9748 - mean_squared_logarithmic_error: 0.2194
Epoch 8/100
7/7 [==============================] - 0s 36ms/step - loss: 0.9501 - mse: 0.9501 - mae: 0.

7/7 [==============================] - 0s 70ms/step - loss: 0.6931 - mse: 0.6931 - mae: 0.6352 - root_mean_squared_error: 0.8325 - mean_squared_logarithmic_error: 0.1845
Epoch 48/100
7/7 [==============================] - 0s 65ms/step - loss: 0.6857 - mse: 0.6857 - mae: 0.6317 - root_mean_squared_error: 0.8281 - mean_squared_logarithmic_error: 0.1773
Epoch 49/100
7/7 [==============================] - 0s 63ms/step - loss: 0.6812 - mse: 0.6812 - mae: 0.6279 - root_mean_squared_error: 0.8254 - mean_squared_logarithmic_error: 0.1717
Epoch 50/100
7/7 [==============================] - 0s 63ms/step - loss: 0.6779 - mse: 0.6779 - mae: 0.6256 - root_mean_squared_error: 0.8233 - mean_squared_logarithmic_error: 0.1689
Epoch 51/100
7/7 [==============================] - 0s 60ms/step - loss: 0.6741 - mse: 0.6741 - mae: 0.6239 - root_mean_squared_error: 0.8211 - mean_squared_logarithmic_error: 0.1673
Epoch 52/100
7/7 [==============================] - 0s 61ms/step - loss: 0.6701 - mse: 0.6701 - ma

7/7 [==============================] - 0s 62ms/step - loss: 0.5193 - mse: 0.5193 - mae: 0.5778 - root_mean_squared_error: 0.7206 - mean_squared_logarithmic_error: 0.1588
Epoch 92/100
7/7 [==============================] - 0s 65ms/step - loss: 0.5132 - mse: 0.5132 - mae: 0.5729 - root_mean_squared_error: 0.7164 - mean_squared_logarithmic_error: 0.1544
Epoch 93/100
7/7 [==============================] - 0s 64ms/step - loss: 0.5096 - mse: 0.5096 - mae: 0.5713 - root_mean_squared_error: 0.7138 - mean_squared_logarithmic_error: 0.1541
Epoch 94/100
7/7 [==============================] - 0s 60ms/step - loss: 0.5091 - mse: 0.5091 - mae: 0.5719 - root_mean_squared_error: 0.7135 - mean_squared_logarithmic_error: 0.1555
Epoch 95/100
7/7 [==============================] - 0s 64ms/step - loss: 0.5040 - mse: 0.5040 - mae: 0.5680 - root_mean_squared_error: 0.7100 - mean_squared_logarithmic_error: 0.1524
Epoch 96/100
7/7 [==============================] - 0s 65ms/step - loss: 0.4992 - mse: 0.4992 - ma

7/7 [==============================] - 0s 68ms/step - loss: 0.8267 - mse: 0.8267 - mae: 0.6627 - root_mean_squared_error: 0.9093 - mean_squared_logarithmic_error: 0.1920
Epoch 34/100
7/7 [==============================] - 1s 90ms/step - loss: 0.8200 - mse: 0.8200 - mae: 0.6609 - root_mean_squared_error: 0.9055 - mean_squared_logarithmic_error: 0.1983
Epoch 35/100
7/7 [==============================] - 1s 71ms/step - loss: 0.8169 - mse: 0.8169 - mae: 0.6594 - root_mean_squared_error: 0.9038 - mean_squared_logarithmic_error: 0.2029
Epoch 36/100
7/7 [==============================] - 1s 71ms/step - loss: 0.8140 - mse: 0.8140 - mae: 0.6580 - root_mean_squared_error: 0.9022 - mean_squared_logarithmic_error: 0.2055
Epoch 37/100
7/7 [==============================] - 0s 68ms/step - loss: 0.8068 - mse: 0.8068 - mae: 0.6567 - root_mean_squared_error: 0.8982 - mean_squared_logarithmic_error: 0.2028
Epoch 38/100
7/7 [==============================] - 1s 86ms/step - loss: 0.7973 - mse: 0.7973 - ma

7/7 [==============================] - 0s 62ms/step - loss: 0.5869 - mse: 0.5869 - mae: 0.5915 - root_mean_squared_error: 0.7661 - mean_squared_logarithmic_error: 0.1512
Epoch 78/100
7/7 [==============================] - 0s 66ms/step - loss: 0.5829 - mse: 0.5829 - mae: 0.5899 - root_mean_squared_error: 0.7635 - mean_squared_logarithmic_error: 0.1506
Epoch 79/100
7/7 [==============================] - 0s 67ms/step - loss: 0.5748 - mse: 0.5748 - mae: 0.5905 - root_mean_squared_error: 0.7581 - mean_squared_logarithmic_error: 0.1521
Epoch 80/100
7/7 [==============================] - 0s 65ms/step - loss: 0.5688 - mse: 0.5688 - mae: 0.5898 - root_mean_squared_error: 0.7542 - mean_squared_logarithmic_error: 0.1527
Epoch 81/100
7/7 [==============================] - 0s 62ms/step - loss: 0.5636 - mse: 0.5636 - mae: 0.5911 - root_mean_squared_error: 0.7507 - mean_squared_logarithmic_error: 0.1553
Epoch 82/100
7/7 [==============================] - 0s 60ms/step - loss: 0.5593 - mse: 0.5593 - ma

In [21]:
%tensorboard --logdir '/home/bavanya/Documents/AST_Training_curves/5dim_xalan2.5/'

In [22]:
total_training_time

[65.28357648849487,
 68.79808473587036,
 62.17009687423706,
 66.30890417098999,
 83.53214120864868,
 71.33627915382385,
 65.72516465187073,
 83.06233072280884,
 83.0792338848114,
 83.05665802955627,
 82.92625689506531,
 67.87500166893005,
 82.98986911773682,
 82.89488673210144,
 82.88748288154602,
 60.851224422454834,
 53.184534311294556,
 51.97022461891174,
 82.66579604148865,
 82.86370325088501,
 59.00881600379944,
 82.85594820976257,
 82.86462569236755,
 82.84283685684204,
 44.60969591140747,
 48.05959701538086,
 47.56049108505249,
 28.472338914871216,
 82.23511052131653,
 52.33531856536865]

In [23]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 69.14354095458984


In [24]:
total_training_loss_history

[{'loss': [1.4572851657867432,
   1.4075956344604492,
   1.3360482454299927,
   1.2365071773529053,
   1.12960684299469,
   1.0244306325912476,
   0.96249920129776,
   0.9397796392440796,
   0.935385525226593,
   0.9352301359176636,
   0.9290281534194946,
   0.9208201766014099,
   0.91282719373703,
   0.9054020643234253,
   0.8980286717414856,
   0.8933711051940918,
   0.8839125633239746,
   0.876776397228241,
   0.8697410821914673,
   0.865175187587738,
   0.8588728308677673,
   0.8494174480438232,
   0.8427711725234985,
   0.8344416618347168,
   0.8283999562263489,
   0.8207765817642212,
   0.8125927448272705,
   0.805534839630127,
   0.7983707785606384,
   0.7903559803962708,
   0.7862238883972168,
   0.7807339429855347,
   0.7701653242111206,
   0.759243905544281,
   0.7517380714416504,
   0.7437329888343811,
   0.7363854646682739,
   0.7274952530860901,
   0.7218466997146606,
   0.7158886790275574,
   0.706707239151001,
   0.701654851436615,
   0.6966299414634705,
   0.68765217065

In [25]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.5401289979616801
avg mae of training: 0.5549141883850097


In [26]:
training_mse

[0.4563276767730713,
 0.5635109543800354,
 0.5599752068519592,
 0.4750760495662689,
 0.41042226552963257,
 1.479270339012146,
 0.4640065133571625,
 0.46030423045158386,
 0.39130517840385437,
 0.5896084308624268,
 0.4999944269657135,
 0.4185093641281128,
 0.563647449016571,
 0.427718847990036,
 0.4626144468784332,
 0.3687991499900818,
 1.479270339012146,
 0.4227139353752136,
 0.5279573202133179,
 0.49398550391197205,
 0.6126857399940491,
 0.481962651014328,
 0.4192349910736084,
 0.469758540391922,
 0.4354056715965271,
 0.44957447052001953,
 0.4776161015033722,
 0.3630327582359314,
 0.4910827577114105,
 0.48849862813949585]

In [27]:
training_mae

[0.5447124242782593,
 0.5827105045318604,
 0.5901709198951721,
 0.5421097278594971,
 0.5148413777351379,
 0.7131011486053467,
 0.5376899838447571,
 0.5428469181060791,
 0.4944409430027008,
 0.595221996307373,
 0.569728434085846,
 0.5283668637275696,
 0.5827335715293884,
 0.5198319554328918,
 0.5557337403297424,
 0.489357054233551,
 0.7131011486053467,
 0.518578290939331,
 0.5747092366218567,
 0.5583611726760864,
 0.5673944354057312,
 0.5554478764533997,
 0.5125196576118469,
 0.5437471866607666,
 0.5227978825569153,
 0.5365163683891296,
 0.5463923215866089,
 0.477732390165329,
 0.5496079921722412,
 0.566922128200531]

In [28]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse of testing: " + str(sum_mse/30))
print("avg mae of testing: " + str(sum_mae/30))
print("avg are of testing: " + str(sum_are/30))

avg mse of testing: 0.7212815225124359
avg mae of testing: 0.6602642714977265
avg are of testing: 0.25743193328380587


In [29]:
test_mse

[0.740576446056366,
 0.7058652639389038,
 0.6797581911087036,
 0.7089570164680481,
 0.7414754033088684,
 0.887417197227478,
 0.705095112323761,
 0.7249408960342407,
 0.7141091227531433,
 0.6422184109687805,
 0.7270113825798035,
 0.7911216020584106,
 0.5904313921928406,
 0.7525684237480164,
 0.718473494052887,
 0.7278121113777161,
 0.887417197227478,
 0.7587324976921082,
 0.6987600922584534,
 0.6549965739250183,
 0.5813682675361633,
 0.7566943168640137,
 0.7245556712150574,
 0.7049955725669861,
 0.7743470072746277,
 0.7214295864105225,
 0.6993045806884766,
 0.709984540939331,
 0.7267482280731201,
 0.6812800765037537]

In [30]:
test_mae

[0.677651047706604,
 0.6613606810569763,
 0.6614311933517456,
 0.6660063862800598,
 0.6753503680229187,
 0.5827814340591431,
 0.6636350750923157,
 0.6692125797271729,
 0.6637125611305237,
 0.654771089553833,
 0.6750431060791016,
 0.6940953135490417,
 0.6330552101135254,
 0.6869186758995056,
 0.6739751696586609,
 0.6698281764984131,
 0.5827814340591431,
 0.6739194989204407,
 0.6678960919380188,
 0.6451091766357422,
 0.6178173422813416,
 0.683029055595398,
 0.6637371182441711,
 0.6645750999450684,
 0.6775238513946533,
 0.6671334505081177,
 0.659839928150177,
 0.6596672534942627,
 0.6697166562080383,
 0.6663541197776794]

In [31]:
test_ARE

[0.24834438,
 0.25165564,
 0.23509933,
 0.25496688,
 0.24503312,
 0.47019866,
 0.22847682,
 0.24834438,
 0.2615894,
 0.19536424,
 0.25496688,
 0.27152318,
 0.12582782,
 0.25165564,
 0.22847682,
 0.26490065,
 0.47019866,
 0.26490065,
 0.23509933,
 0.25827813,
 0.22516556,
 0.24503312,
 0.2615894,
 0.25165564,
 0.27483442,
 0.25827813,
 0.24834438,
 0.2660044,
 0.25827813,
 0.16887417]

In [32]:
total_scores

[[0.740576446056366,
  0.740576446056366,
  0.677651047706604,
  0.8605675101280212,
  0.22232159972190857],
 [0.7058652639389038,
  0.7058652639389038,
  0.6613606810569763,
  0.8401578664779663,
  0.21776849031448364],
 [0.6797581911087036,
  0.6797581911087036,
  0.6614311933517456,
  0.8244745135307312,
  0.21693870425224304],
 [0.7089570164680481,
  0.7089570164680481,
  0.6660063862800598,
  0.8419958353042603,
  0.21590600907802582],
 [0.7414754033088684,
  0.7414754033088684,
  0.6753503680229187,
  0.8610896468162537,
  0.22474078834056854],
 [0.887417197227478,
  0.887417197227478,
  0.5827814340591431,
  0.9420282244682312,
  0.3070068359375],
 [0.705095112323761,
  0.705095112323761,
  0.6636350750923157,
  0.8396994471549988,
  0.21931959688663483],
 [0.7249408960342407,
  0.7249408960342407,
  0.6692125797271729,
  0.851434588432312,
  0.21862465143203735],
 [0.7141091227531433,
  0.7141091227531433,
  0.6637125611305237,
  0.8450497984886169,
  0.2157810479402542],
 [0.6

In [33]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.47241729497909546
median testing mse: 0.7199515402317047


In [34]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.5455523729324341
median testing mae: 0.6661802530288696


In [35]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.251655638217926


In [36]:
accuracies = []
from sklearn.metrics import accuracy_score
for i in range(30):
    accuracies.append(accuracy_score(y, total_predictions[i]))

In [37]:
print("median accuracy is: " + str(statistics.median(accuracies)))

median accuracy is: 0.48344370860927155
